In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Myeloid.2500.250324/pairs.E2G.res.rds"
path.pairs.E2G.reuse = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Myeloid.25000.240612/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/Myeloid/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/RNA_count_matrix/Myeloid.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Myeloid.25000.250329/"
celltype = "Myeloid"

In [3]:
n.cores = 48

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

In [5]:
pairs.E2G.reuse = readRDS(path.pairs.E2G.reuse)

Import ATAC matrix

In [6]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [7]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [8]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [9]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

Prepare SCENT meta data

In [10]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
s1d1_CTATGGCCATAACGGG-1,s1d1_CTATGGCCATAACGGG-1,2157,69,Myeloid,7.676474,0.03198887
s1d1_ACCACATAGGTGTCCA-1,s1d1_ACCACATAGGTGTCCA-1,2128,64,Myeloid,7.662938,0.03007519
s1d1_TGCCGGTAGTCGCAAT-1,s1d1_TGCCGGTAGTCGCAAT-1,1920,67,Myeloid,7.560080,0.03489583
s1d1_CATCCCAGTCTTTATG-1,s1d1_CATCCCAGTCTTTATG-1,3211,127,Myeloid,8.074338,0.03955154
s1d1_TAGTGAGAGCTGCACA-1,s1d1_TAGTGAGAGCTGCACA-1,3568,85,Myeloid,8.179760,0.02382287
s1d1_CCTGTTGGTTATAGCG-1,s1d1_CCTGTTGGTTATAGCG-1,4044,42,Myeloid,8.304990,0.01038576
s1d1_TTTAGCTTCTGGCTTC-1,s1d1_TTTAGCTTCTGGCTTC-1,2728,105,Myeloid,7.911324,0.03848974
s1d1_TGTGAAACAGGATTAA-1,s1d1_TGTGAAACAGGATTAA-1,2800,86,Myeloid,7.937375,0.03071429
s1d1_ACCAGGCTCTTGGATA-1,s1d1_ACCAGGCTCTTGGATA-1,1015,94,Myeloid,6.922644,0.09261084


RUN SCENT prediction

In [11]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
                bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [12]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

Warning message in dir.create(paste(dir.output, "chr", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Myeloid.25000.250329//chr' already exists”


In [13]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

Filter genes which can be reused

In [14]:
map_gene_names_reuse <- function(gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()
    abc_genes.reuse = abc_genes[abc_genes$abc_name == abc_genes$gene_ref_name,]
	return(abc_genes.reuse)
}

In [15]:
abc_genes.reuse = map_gene_names_reuse(gene_gtf_path, abc_genes_path)

In [16]:
dim(abc_genes.reuse)
abc_genes.reuse

[1] 19669     3

abc_name,Ensembl_ID,gene_ref_name
<chr>,<chr>,<chr>
FAM138A,ENSG00000237613,FAM138A
FAM138F,ENSG00000282591,FAM138F
OR4F5,ENSG00000186092,OR4F5
FAM87B,ENSG00000177757,FAM87B
LINC00115,ENSG00000225880,LINC00115
LINC01128,ENSG00000228794,LINC01128
FAM41C,ENSG00000230368,FAM41C
SAMD11,ENSG00000187634,SAMD11
NOC2L,ENSG00000188976,NOC2L


In [17]:
pairs.E2G.reuse.25000 = pairs.E2G.reuse[pairs.E2G.reuse$boot_basic_p < 0.01 & 
                                        pairs.E2G.reuse$TargetGene %in% abc_genes.reuse$gene_ref_name]

In [18]:
names(pairs.E2G.reuse.25000) = pairs.E2G.reuse.25000$PairName

In [19]:
pairs.E2G.reuse.25000

GRanges object with 36081 ranges and 11 metadata columns:
                                  seqnames              ranges strand |
                                     <Rle>           <IRanges>  <Rle> |
      chr1-1001860-1002126_CDK11B     chr1     1001860-1002126      * |
     chr1-10022225-10023589_KIF1B     chr1   10022225-10023589      * |
  chr1-10022225-10023589_SLC25A33     chr1   10022225-10023589      * |
  chr1-100265575-100266826_CDC14A     chr1 100265575-100266826      * |
  chr1-10032414-10033799_SLC25A33     chr1   10032414-10033799      * |
                              ...      ...                 ...    ... .
   chrX-77899134-77899778_CYSLTR1     chrX   77899134-77899778      * |
   chrX-81120681-81122301_SH3BGRL     chrX   81120681-81122301      * |
       chrX-9378667-9379857_TBL1X     chrX     9378667-9379857      * |
    chrX-96722435-96723511_DIAPH2     chrX   96722435-96723511      * |
         chrX-989778-990654_IL3RA     chrX       989778-990654      * |
      

In [20]:
table(pairs.E2G[pairs.E2G$boot_basic_p < 0.01]$PairName %in% pairs.E2G.reuse.25000$PairName)


FALSE  TRUE 
 3486 35150 

In [21]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res.0 = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.01]
  pairs.E2G.chr.keep.0 = pairs.E2G.chr[!pairs.E2G.chr$PairName %in% pairs.E2G.chr.res.0$PairName]
  pairs.E2G.chr.res = pairs.E2G.chr.res.0[!pairs.E2G.chr.res.0$PairName %in% pairs.E2G.reuse.25000$PairName]
  pairs.E2G.chr.keep.1 = pairs.E2G.reuse.25000[pairs.E2G.reuse.25000$PairName %in% pairs.E2G.chr.res.0$PairName]

  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(min(n.cores,floor(length(pairs.E2G.chr.res)/2))))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(min(n.cores,floor(length(pairs.E2G.chr.res)/2)))
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep.0,
                          pairs.E2G.chr.keep.1)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr11"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr12"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr13"


...

[1] "chr8"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Merge results

In [22]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [23]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 318489 ranges and 11 metadata columns:
                                      seqnames            ranges strand |
                                         <Rle>         <IRanges>  <Rle> |
                       chr1.chr1.chr1     chr1   1012920-1014120      * |
                       chr1.chr1.chr1     chr1 10147738-10148559      * |
                       chr1.chr1.chr1     chr1   1019144-1020371      * |
                       chr1.chr1.chr1     chr1   1038601-1039080      * |
                       chr1.chr1.chr1     chr1   1058756-1060110      * |
                                  ...      ...               ...    ... .
  chrX.chrX-77899134-77899778_CYSLTR1     chrX 77899134-77899778      * |
  chrX.chrX-81120681-81122301_SH3BGRL     chrX 81120681-81122301      * |
      chrX.chrX-9378667-9379857_TBL1X     chrX   9378667-9379857      * |
   chrX.chrX-96722435-96723511_DIAPH2     chrX 96722435-96723511      * |
        chrX.chrX-989778-990654_IL3RA     chrX     98

In [24]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,1012920,1014120,UBE2J2,Myeloid,0.15242525,0.06205776,2.456184,0.0140421368,0.01104,1.957031,1,1.957031,0.09000269
chr1,10147738,10148559,UBE4B,Myeloid,0.13261564,0.04710376,2.815394,0.0048717487,0.01552,1.809108,1,1.809108,0.11035832
chr1,1019144,1020371,NADK,Myeloid,-0.10450517,0.03843754,-2.718831,0.0065513095,0.01072,1.969805,-1,-1.969805,0.08870131
chr1,1038601,1039080,ISG15,Myeloid,-0.37455939,0.10286672,-3.641210,0.0002713592,0.01368,1.863914,-1,-1.863914,0.10151280
chr1,1058756,1060110,NADK,Myeloid,-0.12838767,0.04672014,-2.748016,0.0059957130,0.01160,1.935542,-1,-1.935542,0.09284460
chr1,1079374,1080601,CDK11B,Myeloid,0.16449572,0.06110268,2.692120,0.0070999496,0.00712,2.147520,1,2.147520,0.06637324
chr1,109040911,109042568,CLCC1,Myeloid,0.14251012,0.05488633,2.596459,0.0094190065,0.01040,1.982967,1,1.982967,0.08635188
chr1,109091688,109092177,GPSM2,Myeloid,0.28891353,0.10194700,2.833958,0.0045975345,0.01520,1.818156,1,1.818156,0.10814084
chr1,109133440,109134866,GPSM2,Myeloid,0.29310711,0.09617894,3.047519,0.0023073926,0.01088,1.963371,1,1.963371,0.08987810


In [25]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges